<h1>VSS SEB Parsing for Ephemerides</h1>

In [3]:
import pandas as pd
from astroquery.simbad import Simbad

In [4]:
df = pd.DataFrame()

In [5]:
df = pd.read_excel('VSS SEB observed targets.xlsx', sheet_name='Data')

In [6]:
df.head()

,Star,Epoch (HJD),Epoch Error,Period (d),Period Error,O-C Diagram Notes,Recommended Fequency,Last Observed
0,Ant S,2.458519e+06,0.000287,0.648348,2.959964e-07,linear?,yearly,2022-03-08 15:49:44.602
1,Aps DW,2.439210e+06,0.000787,2.312944,9.672835e-08,linear?,yearly,2022-05-09 14:58:03.446
2,Aps II,2.451931e+06,0.149860,0.842229,1.695586e-05,eccentric + apsidal motion?,2 per year,2021-06-23 10:19:15.616
3,Aps MR,2.457159e+06,0.000462,0.527864,1.404435e-07,linear?,yearly,2023-07-26 11:58:57.014
4,Aps NT,2.457208e+06,0.000361,0.294762,5.285139e-08,downward parabola?,yearly,2024-05-23 13:12:24.365


In [78]:
df.columns

Index([                'Star',          'Epoch (HJD)',          'Epoch Error',
                 'Period (d)',         'Period Error',    'O-C Diagram Notes',
       'Recommended Fequency',        'Last Observed',     ('Const', 'VarID')],
      dtype='object')

In [7]:
df['Const', 'VarID'] = df['Star'].str.split()

We parse the weirdness of the input file

In [87]:
objectsdf = pd.DataFrame()
for i in range(len(df)):
    details = df['Const', 'VarID'].iloc[i]
    if (len(details) == 2):
        object_name = details[1] +' '+ details[0]
    if (len(details) > 2):
        if (details[2].lower() == "secondary" or details[2].lower() == "primary"):
            object_name = details[1] +' '+ details[0]
        else:
            object_name = details[1]+details[2]# +' '+ details[0]

    result_table = Simbad.query_object(object_name)
    #print(result_table)
    if (result_table is None):
        print(f'{object_name}: failed to locate')
    else:
        objdf = result_table.to_pandas()
        for colname in df.columns[:-1]:
            #print(colname, df.iloc[i][colname])
            objdf[colname] = df.iloc[i][colname]
        objectsdf = pd.concat([objectsdf, objdf], ignore_index=True)
columns_to_drop = ['RA_PREC', 'DEC_PREC', 'COO_ERR_MAJA', 'COO_ERR_MINA','COO_ERR_ANGLE','COO_QUAL','COO_WAVELENGTH','COO_BIBCODE',	'SCRIPT_NUMBER_ID']
objectsdf = objectsdf.drop(columns=columns_to_drop)

NSV25657: failed to locate


ASASSN-VJ184530.00-593235.0: failed to locate


NSV1389: failed to locate


In [89]:
objectsdf['RA'] = objectsdf['RA'].str.replace(' ',':')
objectsdf['DEC'] = objectsdf['DEC'].str.replace(' ',':')
objectsdf['MAIN_ID'] = objectsdf['MAIN_ID'].str.replace('V* ','')

In [60]:
objdf

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
0,V* W Vol,07 37 40.0546,-69 32 32.383,14,14,0.011,0.0109,90,A,O,2020yCat.1350....0G,1


In [71]:
df

Star                                         Ant S
Epoch (HJD)                         2458518.709393
Epoch Error                               0.000287
Period (d)                                0.648348
Period Error                                   0.0
O-C Diagram Notes                          linear?
Recommended Fequency                        yearly
Last Observed           2022-03-08 15:49:44.602000
(Const, VarID)                            [Ant, S]
Name: 0, dtype: object

In [66]:
type(e)

pandas.core.frame.DataFrame

In [67]:
e

,0
Star,Ant S
Epoch (HJD),2458518.709393
Epoch Error,0.000287
Period (d),0.648348
Period Error,0.0
O-C Diagram Notes,linear?
Recommended Fequency,yearly
Last Observed,2022-03-08 15:49:44.602000
"(Const, VarID)","[Ant, S]"
